#Surface roughness models

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.svm import SVR



# Load data
data = pd.read_csv('Data needed')

# Remove outliers from 'Ra' using the boxplot method
Q1 = data['Ra'].quantile(0.25)
Q3 = data['Ra'].quantile(0.75)
IQR = Q3 - Q1
filtered_data = data[~((data['Ra'] < (Q1 - 1.5 * IQR)) | (data['Ra'] > (Q3 + 1.5 * IQR)))].copy()

# Create 'Cutting Speed' column after outlier removal (assuming tool is in microns and spindle speed in rpm)
filtered_data['Cutting Speed [mm/min]'] = np.pi * (filtered_data['tool'] / 1000) * filtered_data['Spindle Speed [rpm]']

# Retain only necessary columns
selected_columns = ['Cutting Speed [mm/min]', 'Maximum Stepover [micron]', 'Finishing Stepover [micron]', 'Finishing StepDown [micron]', 'Ra']
final_data = filtered_data[selected_columns]


# Define the cases with specified features and target
case1_features = final_data[['Cutting Speed [mm/min]', 'Maximum Stepover [micron]', 'Finishing Stepover [micron]', 'Finishing StepDown [micron]']]
case1_target = final_data['Ra']

case2_features = final_data[['Maximum Stepover [micron]', 'Finishing StepDown [micron]']]
case2_target = final_data['Ra']

case3_features = final_data[['Cutting Speed [mm/min]', 'Maximum Stepover [micron]']]
case3_target = final_data['Ra']

case4_features = final_data[['Cutting Speed [mm/min]', 'Maximum Stepover [micron]', 'Finishing StepDown [micron]']]
case4_target = final_data['Ra']

# Store cases in lists for easy access and future use
feature_cases = [case1_features, case2_features, case3_features, case4_features]
target_cases = [case1_target, case2_target, case3_target, case4_target]


# Split the data with an 85-15 ratio
x_train_case1, x_test_case1, y_train_case1, y_test_case1 = train_test_split(
    case1_features, case1_target, test_size=0.15, random_state=42
)

# Check the shapes of the resulting splits to ensure correct allocation
print("x_train_case1 shape:", x_train_case1.shape)
print("x_test_case1 shape:", x_test_case1.shape)
print("y_train_case1 shape:", y_train_case1.shape)
print("y_test_case1 shape:", y_test_case1.shape)

# Initialize MinMaxScaler for scaling between 0 and 1
scaler_x_case1 = MinMaxScaler()
scaler_y_case1 = MinMaxScaler()

# Fit the scaler on the training data and transform it
x_train_case1_scaled = scaler_x_case1.fit_transform(x_train_case1)
y_train_case1_scaled = scaler_y_case1.fit_transform(y_train_case1.values.reshape(-1, 1))

# Transform the test set using the scaler fitted on the training data
x_test_case1_scaled = scaler_x_case1.transform(x_test_case1)
y_test_case1_scaled = scaler_y_case1.transform(y_test_case1.values.reshape(-1, 1))

# Display the shapes of the scaled data to confirm
print("x_train_case1_scaled shape:", x_train_case1_scaled.shape)
print("x_test_case1_scaled shape:", x_test_case1_scaled.shape)
print("y_train_case1_scaled shape:", y_train_case1_scaled.shape)
print("y_test_case1_scaled shape:", y_test_case1_scaled.shape)

# Initialize the GBR model with the best parameters found by Optuna
gbrcv5case1 = GradientBoostingRegressor(
    n_estimators=90,
    learning_rate=0.03176121316791901,
    max_depth=8,
    min_samples_split=14,
    min_samples_leaf=14,
    subsample=0.9243186123744747,
    random_state=42
)

# Perform 5-fold cross-validation on the training set
cv_scores_gbrcv5case1 = cross_val_score(
    gbrcv5case1,
    x_train_case1_scaled,
    y_train_case1_scaled.ravel(),  # Use .ravel() to convert to 1D array
    cv=5,
    scoring='r2'
)
print("5-Fold Cross-Validation R² Scores for GBR (case1):", cv_scores_gbrcv5case1)
print("Average Cross-Validation R² Score for GBR (case1):", np.mean(cv_scores_gbrcv5case1))

# Fit the model on the entire scaled training set
gbrcv5case1.fit(x_train_case1_scaled, y_train_case1_scaled.ravel())  # Use .ravel() here too


# Split the data with an 85-15 ratio
x_train_case2, x_test_case2, y_train_case2, y_test_case2 = train_test_split(
    case2_features, case2_target, test_size=0.15, random_state=42
)

# Check the shapes of the resulting splits to ensure correct allocation
print("x_train_case2 shape:", x_train_case2.shape)
print("x_test_case2 shape:", x_test_case2.shape)
print("y_train_case2 shape:", y_train_case2.shape)
print("y_test_case2 shape:", y_test_case2.shape)

# Initialize MinMaxScaler for scaling between 0 and 1
scaler_x_case2 = MinMaxScaler()
scaler_y_case2 = MinMaxScaler()

# Fit the scaler on the training data and transform it
x_train_case2_scaled = scaler_x_case2.fit_transform(x_train_case2)
y_train_case2_scaled = scaler_y_case2.fit_transform(y_train_case2.values.reshape(-1, 1))

# Transform the test set using the scaler fitted on the training data
x_test_case2_scaled = scaler_x_case2.transform(x_test_case2)
y_test_case2_scaled = scaler_y_case2.transform(y_test_case2.values.reshape(-1, 1))

# Display the shapes of the scaled data to confirm
print("x_train_case2_scaled shape:", x_train_case2_scaled.shape)
print("x_test_case2_scaled shape:", x_test_case2_scaled.shape)
print("y_train_case2_scaled shape:", y_train_case2_scaled.shape)
print("y_test_case2_scaled shape:", y_test_case2_scaled.shape)


# Initialize the Linear Regression model with a personalized name
linearregressioncv5case2 = LinearRegression()

# Perform 5-fold cross-validation on the training set
cv_scores_linearregressioncv5case2 = cross_val_score(linearregressioncv5case2, x_train_case2_scaled, y_train_case2_scaled, cv=5, scoring='r2')
print("5-Fold Cross-Validation R² Scores for case2:", cv_scores_linearregressioncv5case2)
print("Average Cross-Validation R² Score for case2:", np.mean(cv_scores_linearregressioncv5case2))

# Fit the model on the entire scaled training set
linearregressioncv5case2.fit(x_train_case2_scaled, y_train_case2_scaled)

# Make predictions on the training set to calculate R²
y_train_pred_linearregressioncv5case2 = linearregressioncv5case2.predict(x_train_case2_scaled)
r2_train_linearregressioncv5case2 = r2_score(y_train_case2_scaled, y_train_pred_linearregressioncv5case2)
print("R² Score on Training Set for case2:", r2_train_linearregressioncv5case2)

# Display the coefficients and intercept of the Linear Regression model in a table
coefficients_linearregressioncv5case2 = pd.DataFrame(linearregressioncv5case2.coef_.reshape(-1, 1), index=x_train_case2.columns, columns=['Coefficient'])
coefficients_linearregressioncv5case2.loc['Intercept'] = linearregressioncv5case2.intercept_
print("\nCoefficients of Linear Regression Model for case2 (including Intercept)")
print(coefficients_linearregressioncv5case2)

# Print the model equation
equation = f"Ra = {linearregressioncv5case2.intercept_[0]:.4f}"
for i, col in enumerate(x_train_case2.columns):
    equation += f" + ({linearregressioncv5case2.coef_[0, i]:.4f}) * {col}"
print("\nModel Equation for Linear Regression case2:")
print(equation)


# Split the data with an 85-15 ratio
x_train_case3, x_test_case3, y_train_case3, y_test_case3 = train_test_split(
    case3_features, case3_target, test_size=0.15, random_state=42
)

# Check the shapes of the resulting splits to ensure correct allocation
print("x_train_case3 shape:", x_train_case3.shape)
print("x_test_case3 shape:", x_test_case3.shape)
print("y_train_case3 shape:", y_train_case3.shape)
print("y_test_case3 shape:", y_test_case3.shape)

# Initialize MinMaxScaler for scaling between 0 and 1
scaler_x_case3 = MinMaxScaler()
scaler_y_case3 = MinMaxScaler()

# Fit the scaler on the training data and transform it
x_train_case3_scaled = scaler_x_case3.fit_transform(x_train_case3)
y_train_case3_scaled = scaler_y_case3.fit_transform(y_train_case3.values.reshape(-1, 1))

# Transform the test set using the scaler fitted on the training data
x_test_case3_scaled = scaler_x_case3.transform(x_test_case3)
y_test_case3_scaled = scaler_y_case3.transform(y_test_case3.values.reshape(-1, 1))

# Display the shapes of the scaled data to confirm
print("x_train_case3_scaled shape:", x_train_case3_scaled.shape)
print("x_test_case3_scaled shape:", x_test_case3_scaled.shape)
print("y_train_case3_scaled shape:", y_train_case3_scaled.shape)
print("y_test_case3_scaled shape:", y_test_case3_scaled.shape)


# Initialize the SVR model with the best parameters found by Optuna for case 3
svrcv5case3 = SVR(
    C=0.5565493763103878,
    epsilon=4.086373260966742e-05,
    gamma='auto',
    kernel='rbf'
)

# Perform 5-fold cross-validation on the training set for case 3
cv_scores_svrcv5case3 = cross_val_score(
    svrcv5case3,
    x_train_case3_scaled,
    y_train_case3_scaled.ravel(),  # Use .ravel() to convert to 1D array
    cv=5,
    scoring='r2'
)
print("5-Fold Cross-Validation R² Scores for SVR (case3):", cv_scores_svrcv5case3)
print("Average Cross-Validation R² Score for SVR (case3):", np.mean(cv_scores_svrcv5case3))

# Fit the model on the entire scaled training set for case 3
svrcv5case3.fit(x_train_case3_scaled, y_train_case3_scaled.ravel())  # Use .ravel() here too


# Split the data with an 85-15 ratio
x_train_case4, x_test_case4, y_train_case4, y_test_case4 = train_test_split(
    case4_features, case4_target, test_size=0.15, random_state=42
)

# Check the shapes of the resulting splits to ensure correct allocation
print("x_train_case4 shape:", x_train_case4.shape)
print("x_test_case4 shape:", x_test_case4.shape)
print("y_train_case4 shape:", y_train_case4.shape)
print("y_test_case4 shape:", y_test_case4.shape)

# Initialize MinMaxScaler for scaling between 0 and 1
scaler_x_case4 = MinMaxScaler()
scaler_y_case4 = MinMaxScaler()

# Fit the scaler on the training data and transform it
x_train_case4_scaled = scaler_x_case4.fit_transform(x_train_case4)
y_train_case4_scaled = scaler_y_case4.fit_transform(y_train_case4.values.reshape(-1, 1))

# Transform the test set using the scaler fitted on the training data
x_test_case4_scaled = scaler_x_case4.transform(x_test_case4)
y_test_case4_scaled = scaler_y_case4.transform(y_test_case4.values.reshape(-1, 1))

# Display the shapes of the scaled data to confirm
print("x_train_case4_scaled shape:", x_train_case4_scaled.shape)
print("x_test_case4_scaled shape:", x_test_case4_scaled.shape)
print("y_train_case4_scaled shape:", y_train_case4_scaled.shape)
print("y_test_case4_scaled shape:", y_test_case4_scaled.shape)


# Initialize the SVR model with the best parameters found by Optuna for case 4
svrcv5case4 = SVR(
    C=1.729889999988274,
    epsilon=0.10952300801650905,
    gamma='auto',
    kernel='rbf'
)

# Perform 5-fold cross-validation on the training set for case 4
cv_scores_svrcv5case4 = cross_val_score(
    svrcv5case4,
    x_train_case4_scaled,
    y_train_case4_scaled.ravel(),  # Use .ravel() to convert to 1D array
    cv=5,
    scoring='r2'
)
print("5-Fold Cross-Validation R² Scores for SVR (case4):", cv_scores_svrcv5case4)
print("Average Cross-Validation R² Score for SVR (case4):", np.mean(cv_scores_svrcv5case4))

# Fit the model on the entire scaled training set for case 4
svrcv5case4.fit(x_train_case4_scaled, y_train_case4_scaled.ravel())



#Friction factor Model

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

# Load data
df = pd.read_csv('Data needed')

# Split into train/test (15% test)
train_df_d5, test_df_d5 = train_test_split(df, test_size=0.15, random_state=42)

# Separate features & target
X_train_d5 = train_df_d5.drop(columns=['friction factor'])
X_test_d5  = test_df_d5.drop(columns=['friction factor'])
y_train_d5 = train_df_d5[['friction factor']]
y_test_d5  = test_df_d5[['friction factor']]

# Scale to [0,1]
X_scaler_d5 = MinMaxScaler()
y_scaler_d5 = MinMaxScaler()

X_train_scaled_d5 = X_scaler_d5.fit_transform(X_train_d5)
X_test_scaled_d5  = X_scaler_d5.transform(X_test_d5)
y_train_scaled_d5 = y_scaler_d5.fit_transform(y_train_d5)
y_test_scaled_d5  = y_scaler_d5.transform(y_test_d5)

# Degree-5 Polynomial Features
degree_d5 = 5
poly_d5 = PolynomialFeatures(degree=degree_d5, include_bias=False)
X_tr_poly_d5 = poly_d5.fit_transform(X_train_scaled_d5)
X_te_poly_d5 = poly_d5.transform(X_test_scaled_d5)

poly_model_d5 = LinearRegression()
poly_model_d5.fit(X_tr_poly_d5, y_train_scaled_d5.ravel())

y_tr_pred_d5 = y_scaler_d5.inverse_transform(poly_model_d5.predict(X_tr_poly_d5).reshape(-1,1)).ravel()
y_te_pred_d5 = y_scaler_d5.inverse_transform(poly_model_d5.predict(X_te_poly_d5).reshape(-1,1)).ravel()

r2_tr_d5   = r2_score(train_df_d5['friction factor'], y_tr_pred_d5)
r2_te_d5   = r2_score(test_df_d5['friction factor'],  y_te_pred_d5)
rmse_tr_d5 = np.sqrt(mean_squared_error(train_df_d5['friction factor'], y_tr_pred_d5))
rmse_te_d5 = np.sqrt(mean_squared_error(test_df_d5['friction factor'],  y_te_pred_d5))

print(f"Degree {degree_d5} Polynomial Regression")
print(f" Train → R²: {r2_tr_d5:.4f}, RMSE: {rmse_tr_d5:.4f}")
print(f" Test  → R²: {r2_te_d5:.4f}, RMSE: {rmse_te_d5:.4f}")

#User interface

In [ ]:
# === User interface (Dash) ===
# Purpose: predict surface roughness (µm), friction factor (–), and pressure drop (kPa/cm)
# from machining inputs and channel geometry. Models/scalers must be loaded beforehand.

from dash import Dash, html, dcc, Input, Output, State
import dash
import numpy as np
import traceback

# Single-page Dash app (callbacks can reference components created later)
app = Dash(__name__, suppress_callback_exceptions=True)

# ---------------------------------------------------------------------
# Prerequisites (expected to exist in the runtime; not loaded here)
# ---------------------------------------------------------------------
# gbrcv5case1, linearregressioncv5case2, svrcv5case3, svrcv5case4
# scaler_x_case1, scaler_y_case1, scaler_x_case2, scaler_y_case2,
# scaler_x_case3, scaler_y_case3, scaler_x_case4, scaler_y_case4
# poly_model_d5, poly_d5, X_scaler_d5, y_scaler_d5
# ---------------------------------------------------------------------

# Default values shown on first load (units in labels)
DEFAULTS = {
    "cutting-speed": 15000,     # mm/min
    "max-stepover": 10,          # µm
    "finishing-stepover": 9,     # µm
    "finishing-stepdown": 8,     # µm
    "height": 0.02,              # mm
    "width": 0.02,               # mm
    "reynolds": 10,              # –
    "relative-roughness": 5,     # % (only used if rr-mode="input")
    "rr-mode": "input",          # "input" or "predicted"
}

# Simple color palette & styles (no external CSS)
COLOR_PRIMARY = "#07617D"; COLOR_TEXT = "#1E3E62"; COLOR_TEXT2 = "#ffffff"
COLOR_BUTTON_BG = "#2C061F"; COLOR_OUTPUT_BG = "#374045"; COLOR_HEADER_BG = "#353940"
white = "#ffffff"

LABEL_STYLE = {"color": COLOR_TEXT, "fontSize": "18px", "display": "block", "marginBottom": "4px"}
INPUT_STYLE = {"width": "100%", "textAlign": "center", "fontSize": "18px",
               "border": "1px solid #ccc", "padding": "8px", "minHeight": "24px", "boxSizing": "border-box"}
FIELD_STYLE = {"marginBottom": "8px"}
RESULT_BOX_STYLE = {"border": "1px solid #ccc", "padding": "8px", "minHeight": "24px",
                    "fontSize": "18px", "marginBottom": "0px", "boxSizing": "border-box"}

TOP_GRID = {"display": "grid", "gridTemplateColumns": "repeat(4, minmax(180px, 1fr))",
            "columnGap": "16px", "rowGap": "10px", "alignItems": "start"}
RESULTS_ROW_GRID = {"display": "grid", "gridTemplateColumns": "repeat(3, minmax(220px, 1fr))",
                    "columnGap": "16px", "rowGap": "6px", "alignItems": "end"}

def stacked_number_field(label, comp_id, value, step, min_=None, max_=None):
    """Reusable numeric input (label above field)."""
    return html.Div([
        html.Label(label, style=LABEL_STYLE),
        dcc.Input(id=comp_id, type="number", value=value, step=step, min=min_, max=max_, style=INPUT_STYLE)
    ], style=FIELD_STYLE)

def validate_inputs(cs, ms, fs, fd, h_mm, w_mm, Re, rr_pct):
    """Basic range checks (keeps UI robust; units match labels)."""
    if cs is None or cs < 15000 or cs > 130000: return False, "Cutting Speed must be between 15000 and 130000."
    if ms is None or ms < 10 or ms > 200:       return False, "Maximum Stepover must be between 10 and 200."
    if fs is None or fs >= ms:                  return False, "Finishing Stepover must be less than Maximum Stepover."
    if fs < 5:                                  return False, "Finishing Stepover must be more than 5."
    if fd is None or fd < 5 or fd > 200:        return False, "Finishing StepDown must be between 5 and 200."
    if h_mm is None or w_mm is None or Re is None or rr_pct is None:
        return False, "Channel height, width, Reynolds number, and Relative Roughness are required."
    if h_mm <= 0 or w_mm <= 0 or Re <= 0:       return False, "Channel height, width, and Reynolds number must be positive."
    if h_mm < 0.02 or h_mm > 5.0:               return False, "Channel height must be between 0.02 mm and 5 mm."
    if w_mm < 0.02 or w_mm > 5.0:               return False, "Channel width must be between 0.02 mm and 5 mm."
    if rr_pct < 0:                               return False, "Relative roughness must be zero or positive."
    return True, ""

def predict_models(cutting_speed, max_stepover, finishing_stepover, finishing_stepdown):
    """
    Predict Ra (µm) using four trained models.
    - Input vector order matches each case.
    - Weighted average uses inverse test RMSE (from manuscript) for the ensemble.
    """
    input_data = np.array([[cutting_speed, max_stepover, finishing_stepover, finishing_stepdown]])
    # Prepare each case’s feature subset via its own scaler
    X1 = scaler_x_case1.transform(input_data)
    X2 = scaler_x_case2.transform(input_data[:, [1, 3]])
    X3 = scaler_x_case3.transform(input_data[:, [0, 1]])
    X4 = scaler_x_case4.transform(input_data[:, [0, 1, 3]])

    # Model predictions (scaled) → inverse-transform to µm
    p1 = gbrcv5case1.predict(X1); p2 = linearregressioncv5case2.predict(X2)
    p3 = svrcv5case3.predict(X3); p4 = svrcv5case4.predict(X4)
    r1 = scaler_y_case1.inverse_transform(p1.reshape(-1,1))[0,0]
    r2 = scaler_y_case2.inverse_transform(p2.reshape(-1,1))[0,0]
    r3 = scaler_y_case3.inverse_transform(p3.reshape(-1,1))[0,0]
    r4 = scaler_y_case4.inverse_transform(p4.reshape(-1,1))[0,0]

    # Inverse-RMSE weights (lower RMSE → higher weight)
    rmse = np.array([0.168, 0.185, 0.17, 0.186]); w = (1/rmse); w = w/w.sum()
    final = float(np.dot(w, [r1, r2, r3, r4]))
    return final, {"GBR (0.168)": r1, "LinReg (0.185)": r2, "SVR1 (0.170)": r3, "SVR2 (0.186)": r4}

def calc_f_dp(Re, Dh, AR, rr, rho=998.0, mu=1.005e-3):
    """
    Compute friction factor (f) and pressure drop per cm (kPa/cm).
    - Branching:
        rr ≤ 1%                → laminar 64/Re
        in ML domain           → polynomial degree-5 model (scaled)
        otherwise              → empirical correction: f = (64/Re)*(1 + 30*rr^1.35)
    - U from Re definition; Δp for a straight channel.
    """
    if 0 <= rr <= 0.01:
        f = 64.0 / Re; method = "Used 64/Re (rr ≤ 1%)"
    elif 0.05 <= rr <= 0.075 and 0.025 <= AR <= 4 and 0 < Re <= 60 and 2e-5 <= Dh <= 8e-5:
        X_in = X_scaler_d5.transform([[Re, rr, Dh]])
        Xp = poly_d5.transform(X_in)
        f = y_scaler_d5.inverse_transform(poly_model_d5.predict(Xp).reshape(-1,1))[0,0]
        method = "Used polynomial ML model (d=5)"
    else:
        f0 = 64.0/Re; f = f0*(1 + 30*rr**1.35); method = "Used f = f0·(1+30·rr^1.35)"

    # Mean velocity from Re; Δp per cm (kPa/cm)
    U = Re*mu/(rho*Dh)
    dp_kpa_per_cm = f*0.01*rho*U**2/(2*Dh*1000)
    return f, dp_kpa_per_cm, method, U

# ---------------- UI LAYOUT ----------------
# Uses an assets/ folder image (optional)
logo_url = app.get_asset_url('cropped-dxbiotech-logo.png')

app.layout = html.Div([

    # Header bar with logo, title, and contact link
    html.Div([
        html.Img(src=logo_url,
                 style={"height":"80px","position":"absolute","left":"20px","top":"50%","transform":"translateY(-50%)"}),
        html.H1("MLMicroMilling",
                style={"color":white,"margin":0,"fontSize":"36px","textAlign":"center","lineHeight":"80px","width":"100%"}),
        html.A(
            "Info: anorouzi24@ku.edu.tr",
            href="mailto:anorouzi24@ku.edu.tr?subject=MLMicroMilling%20support",
            title="Contact support",
            style={"position":"absolute","right":"16px","bottom":"6px","fontSize":"12px","color":"#eaeff3",
                   "textDecoration":"none", "opacity":"1.0"}
        )
    ], style={"backgroundColor":COLOR_HEADER_BG,"height":"80px","position":"relative","width":"100%","marginBottom":"8px"}),

    # Main card
    html.Div([

        # Inputs + buttons
        html.Div([
            html.Div([
                stacked_number_field("Cutting Speed (mm/min):", "cutting-speed", DEFAULTS["cutting-speed"], 500, 15000, 130000),
                stacked_number_field("Max Stepover (µm):", "max-stepover", DEFAULTS["max-stepover"], 1, 10, 200),
                stacked_number_field("Channel Height (mm):", "height", DEFAULTS["height"], 0.001, 0.02, 5.0),
                stacked_number_field("Channel Width (mm):", "width", DEFAULTS["width"], 0.001, 0.02, 5.0),
                stacked_number_field("Finish Stepover (µm):", "finishing-stepover", DEFAULTS["finishing-stepover"], 1, 5, 199),
                stacked_number_field("Finish StepDown (µm):", "finishing-stepdown", DEFAULTS["finishing-stepdown"], 1, 5, 200),
                stacked_number_field("Reynolds #:", "reynolds", DEFAULTS["reynolds"], 1, 10, 2300),
                stacked_number_field("Rel. Roughness (%):", "relative-roughness", DEFAULTS["relative-roughness"], 0.1, 0, 25),
            ], style=TOP_GRID),

            # RR source selection + action buttons
            html.Div([
                html.Div([
                    html.Span("RR mode:", style={"fontSize":"16px","fontWeight":"600","marginRight":"8px"}),
                    dcc.RadioItems(
                        id="rr-mode",
                        options=[{"label":"Use Input RR","value":"input"},
                                 {"label":"Use Predicted SR","value":"predicted"}],
                        value=DEFAULTS["rr-mode"],
                        labelStyle={"display":"inline-block","marginRight":"10px"},
                        style={"fontSize":"16px"}
                    ),
                ], style={"display":"flex","alignItems":"center","flexWrap":"wrap"}),

                html.Div([
                    html.Button("Submit", id="submit-button", n_clicks=0,
                                style={"backgroundColor":COLOR_BUTTON_BG,"color":white,"fontSize":"16px","lineHeight":"1"}),
                    html.Button("Reset", id="reset-button", n_clicks=0,
                                style={"backgroundColor":"#6c757d","color":white,"fontSize":"16px","lineHeight":"1","marginLeft":"8px"})
                ], style={"marginLeft":"auto"})
            ], style={"display":"flex","alignItems":"center","gap":"12px","marginTop":"2px"}),

            # Validation message
            html.Div(id="input-error", style={"color":"red","fontSize":"14px","minHeight":"0", "marginTop":"0"})
        ], style={"padding":"8px 16px 0", "boxSizing":"border-box"}),

        html.Hr(style={"border":"0","borderTop":"2px solid #02383C","margin":"4px 16px 0"}),

        # Results panel (text-only)
        html.Div([
            html.H3("Results", style={"color":COLOR_PRIMARY, "fontSize":"20px","margin":"6px 16px"}),

            dcc.Loading(type="default", children=html.Div([
                html.Div([html.Label("Predicted Surface Roughness (µm):", style=LABEL_STYLE),
                          html.Div(id="sr-output", style=RESULT_BOX_STYLE)]),
                html.Div([html.Label("Friction factor:", style=LABEL_STYLE),
                          html.Div(id="f-output", style=RESULT_BOX_STYLE)]),
                html.Div([html.Label("Pressure drop (kPa/cm):", style=LABEL_STYLE),
                          html.Div(id="dp-output", style=RESULT_BOX_STYLE)]),
            ], style={**RESULTS_ROW_GRID, "padding":"0 16px"})),

            html.Div(
                id="log-output",
                style={"backgroundColor":COLOR_OUTPUT_BG,"color":COLOR_TEXT2,
                       "padding":"8px","minHeight":"48px","maxHeight":"72px",
                       "overflowY":"auto","border":"1px solid #ccc","margin":"6px 16px 8px"}
            )
        ])
    ], style={"width":"95%","margin":"0 auto","border":"2px solid #02383C","borderRadius":"5px","backgroundColor":white}),
])

# ---------------- CALLBACK ----------------
@app.callback(
    Output("sr-output", "children"),
    Output("f-output", "children"),
    Output("dp-output", "children"),
    Output("log-output", "children"),
    Output("input-error", "children"),
    Output("cutting-speed", "value"),
    Output("max-stepover", "value"),
    Output("finishing-stepover", "value"),
    Output("finishing-stepdown", "value"),
    Output("height", "value"),
    Output("width", "value"),
    Output("reynolds", "value"),
    Output("relative-roughness", "value"),
    Output("rr-mode", "value"),
    Input("submit-button", "n_clicks"),
    Input("reset-button", "n_clicks"),
    State("cutting-speed","value"),
    State("max-stepover","value"),
    State("finishing-stepover","value"),
    State("finishing-stepdown","value"),
    State("height","value"),
    State("width","value"),
    State("reynolds","value"),
    State("relative-roughness","value"),
    State("rr-mode","value")
)
def submit_or_reset(submit_n, reset_n, cs, ms, fs, fd, h_mm, w_mm, Re, rr_pct, rr_mode):
    """
    Handle Submit/Reset:
    - Reset → restore DEFAULTS
    - Submit → validate → predict Ra → choose RR source → compute f and Δp → log details
    """
    trig = dash.callback_context.triggered[0]["prop_id"].split(".")[0] if dash.callback_context.triggered else None

    # Reset button: return blanks and defaults
    if trig == "reset-button":
        return ("", "", "", [], "",
                DEFAULTS["cutting-speed"], DEFAULTS["max-stepover"],
                DEFAULTS["finishing-stepover"], DEFAULTS["finishing-stepdown"],
                DEFAULTS["height"], DEFAULTS["width"], DEFAULTS["reynolds"],
                DEFAULTS["relative-roughness"], DEFAULTS["rr-mode"])

    # Before first submit, keep current values unchanged
    if not submit_n:
        return "", "", "", [], "", cs, ms, fs, fd, h_mm, w_mm, Re, rr_pct, rr_mode

    # Input validation
    valid, msg = validate_inputs(cs, ms, fs, fd, h_mm, w_mm, Re, rr_pct)
    if not valid:
        return "", "", "", [], msg, cs, ms, fs, fd, h_mm, w_mm, Re, rr_pct, rr_mode

    # Geometry (m) & properties (water @ ~20–25 °C)
    h = h_mm * 1e-3; w = w_mm * 1e-3
    rho = 998.0; mu = 1.005e-3
    Dh = 2*h*w/(h+w)       # hydraulic diameter (m)
    AR = h/w               # aspect ratio

    # Surface roughness prediction (µm)
    try:
        final_sr_um, _ = predict_models(cs, ms, fs, fd)
    except Exception:
        traceback.print_exc()
        return "", "", "", [], "Prediction error: check models/scalers.", cs, ms, fs, fd, h_mm, w_mm, Re, rr_pct, rr_mode
    sr_text = f"{final_sr_um:.3f} µm"

    # Relative roughness: predicted from SR (µm → m, divide by Dh) or user input (%)
    rr_pred = (final_sr_um * 1e-6) / Dh
    rr_used = rr_pred if rr_mode == "predicted" else (rr_pct / 100.0)

    # f and Δp per cm (kPa/cm)
    try:
        f_val, dp_val, method, U = calc_f_dp(Re, Dh, AR, rr_used, rho=rho, mu=mu)
    except Exception:
        traceback.print_exc()
        return sr_text, "", "", [], "f/Δp error: check polynomial model objects and ranges.", cs, ms, fs, fd, h_mm, w_mm, Re, rr_pct, rr_mode

    # Short log grid (helps interpret the numbers)
    log_items = [
        f"{'Calculated' if rr_mode=='predicted' else 'Input'} Relative Roughness = {rr_used*100:.2f} %",
        f"Aspect Ratio = {AR:.3f}",
        f"Hydraulic Diameter = {Dh:.5f} m",
        "Pressure drop is for straight channel."
    ]
    log_grid = html.Div([html.Div(item) for item in log_items],
                        style={"display":"grid","gridTemplateColumns":"repeat(2, minmax(200px, 1fr))",
                               "columnGap":"16px","rowGap":"6px"})

    return sr_text, f"{f_val:.5f}", f"{dp_val:.2f}", log_grid, "", cs, ms, fs, fd, h_mm, w_mm, Re, rr_pct, rr_mode

# Local dev run (set debug=False for production)
if __name__ == "__main__":
    app.run(debug=True)
